In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#change the file location according to your file location in google drive
import os
os.chdir('/content/drive/My Drive/Final_Project_CS498/Filtered_Data_from_Illinois/')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
all_data=pd.read_csv("AllData_Filtered_Merged_ConsolidatedCategories.csv", index_col=0)

In [ ]:
all_data.columns

Index(['medwid', 'no_lanes', 'lanewid', 'aadt', 'trf_cntl', 'curv_rad',
       'begmp', 'seg_lng', 'endmp', 'access', 'oneway', 'spd_limt', 'med_type',
       'surf_typ', 'cnty_rte', 'rururb', 'rodwycls', 'AccCount',
       'Severity_PDO', 'Severity_Inj_TypeA', 'Severity_Inj_TypeB',
       'Severity_Inj_TypeC', 'Severity_Fatal', 'Severity_Unknown',
       'Weather_Clear', 'Weather_Rain', 'Weather_Snow', 'Weather_Fog',
       'Weather_Hail', 'Weather_CrossWind', 'Weather_Other', 'Weather_Unknown',
       'Year', 'curv_rad_binary', 'med_type_binary', 'Surface_Category',
       'surf_cat'],
      dtype='object')

In [ ]:
df= all_data.loc[:,['aadt', 'no_lanes', 'access', 'curv_rad', 'oneway','spd_limt', 'rururb', 'cnty_rte']]

In [ ]:
testing= all_data[all_data['spd_limt']== 0]

In [ ]:
training= all_data[all_data['spd_limt']!= 0]

In [ ]:
print(training.no_lanes.unique())

[ 2  4  1  3  5  6  8  7  9 10 12 11]


In [ ]:
X = training.loc[:,['aadt', 'no_lanes', 'access','curv_rad', 'oneway', 'rururb', 'cnty_rte']]
y= training.loc[:,['spd_limt']]

In [ ]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

RF

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
classifier = RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=15,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)



In [ ]:
accuracy= accuracy_score(y_test, y_pred)
print(accuracy)

0.8681951273410309


In [ ]:
y_newpred= classifier.predict(scaler.transform(testing.loc[:,['aadt', 'no_lanes', 'access','curv_rad', 'oneway', 'rururb', 'cnty_rte']]))

In [ ]:
print(y_newpred)

[55 45 55 ... 55 55 55]


In [ ]:
y_newpred.mean()

49.21009609444059

In [ ]:
testing.loc[:,'spd_limt']=y_newpred

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
print(testing.columns.to_list())

['medwid', 'no_lanes', 'lanewid', 'aadt', 'trf_cntl', 'curv_rad', 'begmp', 'seg_lng', 'endmp', 'access', 'oneway', 'spd_limt', 'med_type', 'surf_typ', 'cnty_rte', 'rururb', 'rodwycls', 'AccCount', 'Severity_PDO', 'Severity_Inj_TypeA', 'Severity_Inj_TypeB', 'Severity_Inj_TypeC', 'Severity_Fatal', 'Severity_Unknown', 'Weather_Clear', 'Weather_Rain', 'Weather_Snow', 'Weather_Fog', 'Weather_Hail', 'Weather_CrossWind', 'Weather_Other', 'Weather_Unknown', 'Year', 'curv_rad_binary', 'med_type_binary', 'Surface_Category', 'surf_cat']


In [ ]:
all_data1=pd.concat([training, testing], ignore_index=True, sort=True)

In [ ]:
all_data1.sort_values(by=['Year'],inplace=True)

In [ ]:
all_data1.to_csv("all_data_with_spt_limt.csv")

Logistic

In [ ]:
def try_logistic(X_train, y_train, X_test, y_test, features):
    X_train, X_test = X_train[features], X_test[features]
    
    pipe = make_pipeline(StandardScaler(), LogisticRegression(multi_class='multinomial', solver='lbfgs'))
    model=pipe.fit(X_train, y_train)  # apply scaling on training data

    
    #model = LR.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    LR_accuracy=  accuracy_score(y_test, y_predict)
    models_initial = pd.DataFrame({
        'Model'       : ['Logistic'],
        'Accuracy'    : [ LR_accuracy]
        }, columns = ['Model', 'Accuracy'])
    print(models_initial)
    #return LR_roc, LR_accuracy, LR_precision, LR_recall, LR_f1, model

In [ ]:
try_logistic(X_train, y_train.values.ravel(), X_test, y_test.values.ravel(), X_train.columns.to_list() )

AttributeError: ignored